#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 2, 15)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,30,S,WHART,"10,000",12.80,1,DOS,SET


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
2,B,CPNCG,0,HD,SET,"10,000",12.40,12.40,-0.30,-2.36%,2,Buy,"124,000.00"
3,B,GLOBAL,-4,ROUND,SET50,"8,000",19.40,19.80,-0.10,-0.50%,2,Buy,"155,200.00"
13,B,SYNEX,-3,RD15pct,SET100,"3,000",24.50,24.80,-1.45,-5.52%,2,Buy,"73,500.00"
14,B,TISCO,-3,HD,SET50,"2,000",98.00,98.75,-0.25,-0.25%,2,Buy,"196,000.00"
18,S,DIF,1,CUT,SET,"10,000",14.30,14.20,0.00,0.00%,2,Sell,"143,000.00"
22,S,PTT,3,5pct,SET50,"3,000",41.25,40.50,0.25,+0.62%,2,Sell,"123,750.00"


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,CPNCG,0,HD,SET,"10,000",12.40,12.40,-0.30,-2.36%,2,Buy,"124,000.00"
B,ASP,-4,6L,SET,"30,000",3.70,3.78,-0.04,-1.05%,1,Buy,"111,000.00"
B,BGC,-4,DOS,SET,"10,000",10.00,10.40,-0.10,-0.95%,1,Buy,"100,000.00"
B,GLOBAL,-4,ROUND,SET50,"8,000",19.40,19.80,-0.10,-0.50%,2,Buy,"155,200.00"
B,KKP,-9,3L,SET100,"3,000",68.00,70.25,-0.50,-0.71%,0,Buy,"204,000.00"


### End of Tables in the process

### Print to verify before upload file

In [65]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [66]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [67]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,GLOBAL,"8,000",19.40,"155,200.00",ROUND,SET50,2
1,B,MEGA,"4,500",43.00,"193,500.00",1L,SET100,2
2,B,TISCO,"2,000",98.00,"196,000.00",HD,SET50,2
3,S,DIF,"10,000",14.30,"143,000.00",CUT,SET,2
4,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2
5,S,TSTH,"75,000",1.43,"107,250.00",CUT,SET,2


In [68]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    544700.0
S    374000.0
Name: amount, dtype: float64

In [69]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,GLOBAL,"8,000",19.40,"155,200.00",ROUND,SET50,2
1,B,MEGA,"4,500",43.00,"193,500.00",1L,SET100,2
2,B,TISCO,"2,000",98.00,"196,000.00",HD,SET50,2


In [70]:
cash = 556_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(556000, 544700.0, 11300.0)

In [71]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
3,S,DIF,"10,000",14.30,"143,000.00",CUT,SET,2
4,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2
5,S,TSTH,"75,000",1.43,"107,250.00",CUT,SET,2


In [72]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(556000, 374000.0, 930000.0)

### After call ord-log (must call everytime that orders change)

In [73]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [74]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [75]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
0,B,ASP,-3,6L,SET,"30,000",3.70,3.76,-0.02,-0.53%,1,Buy,"111,000.00"
5,B,MEGA,0,1L,SET100,"4,500",43.00,43.00,-0.25,-0.58%,2,Buy,"193,500.00"
14,B,TISCO,-2,HD,SET50,"2,000",98.00,98.50,-0.25,-0.25%,2,Buy,"196,000.00"
15,B,TQM,-3,3L,SET100,"4,500",45.00,45.75,-0.25,-0.54%,1,Buy,"202,500.00"


In [76]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [77]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
18,S,DIF,1,CUT,SET,"10,000",14.30,14.20,0.00,0.00%,2,Sell,"143,000.00"
22,S,PTT,3,5pct,SET50,"3,000",41.25,40.50,0.00,0.00%,2,Sell,"123,750.00"
25,S,TSTH,1,CUT,SET,"75,000",1.43,1.42,0.01,+0.71%,2,Sell,"107,250.00"


In [78]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
0,B,ASP,-3,6L,SET,30000,3.70,3.76,-0.02,-0.53%,1,Buy,111000.0
5,B,MEGA,0,1L,SET100,4500,43.00,43.00,-0.25,-0.58%,2,Buy,193500.0
14,B,TISCO,-2,HD,SET50,2000,98.00,98.50,-0.25,-0.25%,2,Buy,196000.0
15,B,TQM,-3,3L,SET100,4500,45.00,45.75,-0.25,-0.54%,1,Buy,202500.0
18,S,DIF,1,CUT,SET,10000,14.30,14.20,0.00,0.00%,2,Sell,143000.0
22,S,PTT,3,5pct,SET50,3000,41.25,40.50,0.00,0.00%,2,Sell,123750.0
25,S,TSTH,1,CUT,SET,75000,1.43,1.42,0.01,+0.71%,2,Sell,107250.0


In [79]:
df[mb | ms].shape[0]

7

In [80]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
15,TQM,Buy,"202,500.00"
14,TISCO,Buy,"196,000.00"
5,MEGA,Buy,"193,500.00"
18,DIF,Sell,"143,000.00"
22,PTT,Sell,"123,750.00"


### Select source of orders between these two

In [81]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent
0,Buy,ASP,"30,000",3.70,1,-3,3.76,-0.02,-0.53%
5,Buy,MEGA,"4,500",43.00,2,0,43.00,-0.25,-0.58%
14,Buy,TISCO,"2,000",98.00,2,-2,98.50,-0.25,-0.25%
15,Buy,TQM,"4,500",45.00,1,-3,45.75,-0.25,-0.54%
18,Sell,DIF,"10,000",14.30,2,1,14.20,0.00,0.00%
22,Sell,PTT,"3,000",41.25,2,3,40.50,0.00,0.00%
25,Sell,TSTH,"75,000",1.43,2,1,1.42,0.01,+0.71%


In [82]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent
3,Buy,GLOBAL,"8,000",19.40,2,-11,20.50,0.70,+3.54%
5,Buy,MEGA,"4,500",43.00,2,0,43.00,-0.25,-0.58%
14,Buy,TISCO,"2,000",98.00,2,-2,98.50,-0.25,-0.25%
18,Sell,DIF,"10,000",14.30,2,1,14.20,0.00,0.00%
22,Sell,PTT,"3,000",41.25,2,3,40.50,0.00,0.00%
25,Sell,TSTH,"75,000",1.43,2,1,1.42,0.01,+0.71%


In [83]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process